In [5]:
import os
import re
import sys
import math
from tqdm import trange
import matplotlib.pyplot as plt
from numba import jit, njit, cuda, vectorize
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

cuda.detect()

Found 1 CUDA devices
id 0    b'NVIDIA GeForce MX130'                 [SUPPORTED (DEPRECATED)]
                      Compute Capability: 5.0
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-8d855ea3-c829-4ee9-ffc2-f69c7a034726
                                Watchdog: Enabled
                            Compute Mode: WDDM
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [3]:
# CONSTANTS

NO_KEYS = 256

In [5]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

inv_sbox = [
   0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
   0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
   0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
   0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
   0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
   0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
   0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
   0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
   0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
   0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
   0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
   0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
   0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
   0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
   0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
   0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D
]


sBox = np.transpose(np.reshape(sbox, (16, 16))).flatten()


inv_sBox = np.reshape(inv_sbox, (16, 16)).flatten()


HW = np.array([ bin(k).count("1") for k in np.arange(0,256) ]).astype(int).flatten()


def int_2_hex_string(input):
    return (("0x%0.2X" % input).split('x')[-1])


def sub_bytes(next):
    return sBox[next] 


def inv_sub_bytes(next: int) -> int:
    return inv_sBox[next] 
    

def add_round_key(key, next):
    return (key ^ next)


def forward_lookup(next, key):
    return(add_round_key(sub_bytes(next), key)) 


def generate_lookup_table():
    lookup_table = np.zeros((NO_KEYS,NO_KEYS))
    for ct_prev in range(NO_KEYS):
        for key in range(NO_KEYS):
            lookup_table[ct_prev,key] = forward_lookup(ct_prev, key)
    return lookup_table.astype(int)


def inverse_lookup(key_8: int, ct_8: int) -> int:
    global FORWARD_LOOKUP_TABLE
    column = FORWARD_LOOKUP_TABLE[:, key_8]
    return np.where(column == ct_8)[0][0] 


def generate_inverse_lookup_table():
    inverse_lookup_table = np.zeros((NO_KEYS,NO_KEYS))
    for key in range(NO_KEYS):
        for ct_8 in range(NO_KEYS):
            inverse_lookup_table[key, ct_8] = inv_sub_bytes(add_round_key(key, ct_8)) 
    
    return inverse_lookup_table.astype(int) 


# def generate_inverse_lookup_table():
#     inverse_lookup_table = np.zeros((NO_KEYS,NO_KEYS))
#     for key in range(NO_KEYS):
#         for ct_8 in range(NO_KEYS):
#             inverse_lookup_table[key, ct_8] = inverse_lookup(key, ct_8)
    
#     return inverse_lookup_table.astype(int) 


# Generate Forward Lookup Table
FORWARD_LOOKUP_TABLE = generate_lookup_table()

# Generate Inverse Lookup Table
INVERSE_LOOKUP_TABLE = generate_inverse_lookup_table()

# Inverse Mapping
CT_2_PREVCT_MAPPING = np.array([0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12, 1, 6, 11]).astype(int).flatten()


In [6]:
# Data Access Functions

def get_power_from_raw_simulation(filename, trace_indx, no_points):
    file          = open(filename, "r")
    file_line_arr = file.readlines()
    
    pwr_trc = []
    begin   = trace_indx * no_points
    end     = begin + no_points

    for i in range(begin, end):
        indx = (i*7) + 23
        pwr_trc.append( [float(file_line_arr[indx].strip('1'))] )

    file.close()

    return np.array(pwr_trc)


def get_power_from_real_data(filename):
    file          = open(filename, "r")
    file_line_arr = file.readlines()
    
    pwr_trc = []
    
    for line in file_line_arr:
        pwr_trc.append( [ float(k) for k in line.split()] ) 

    file.close()

    return np.array(pwr_trc) 


def get_ct(filename):
    file          = open(filename, "r")
    file_line_arr = file.readlines()

    ct_arr = []

    for ct in file_line_arr:
        temp  =  "".join(ct.split())
        temp2 = [temp[i:i+2] for i in range(0, len(temp), 2)]
        temp  = [int(k,16) for k in temp2]
        
        ct_arr.append(temp)

    
    file.close

    return np.array(ct_arr)

In [55]:
# Statistics

def mean(X):
    return np.mean(X,axis=0)


def sdev(X, X_bar):
    return np.sqrt(np.sum((X-X_bar)**2,axis=0))

@cuda.jit
def sdev_cuda(X, X_bar, result):
    i, j = cuda.grid(2)
    
    if i < X.shape[0] and j < X.shape[1]:
        temp = 0.
        for k in range(X.shape[0]):
            temp += (X[k,j] - X_bar[j])**2
        
        result[j] = math.sqrt(temp)


def cov(X, X_bar, Y, Y_bar):
    return np.sum((X-X_bar)*(Y-Y_bar),axis=0)

In [107]:
# Get Data

NO_PWR_TRC = 3000
PWR_TRC_INDX = 0

power_trace = get_power_from_raw_simulation("../data/simulation/power_5000_3.out", PWR_TRC_INDX, NO_PWR_TRC)
cipher_texts = get_ct("../data/simulation/55nm_5000_1.txt") 

# power_trace = get_power_from_real_data("../data/real/sumesh_data/data_x3000.txt") 
# cipher_texts = get_ct("../data/real/sumesh_data/ct_x3000.txt")

print(cipher_texts[0])

[171  34 181 213 199 161 117  35  16 126  15 225 183  70 115  10]


In [128]:
# %%timeit
# Get Statistics (Mean and Standard Deviation) of Power Traces

NO_POINT_PER_TRACE = power_trace[0].size

mean_power_trace_1 = mean(power_trace)
sdev_power_trace_1 = sdev(power_trace, mean_power_trace_1) 

#print("Points per Trace: ", NO_POINT_PER_TRACE) 

mean_power_trace = []
sdev_power_trace = [] 

for col in range(NO_POINT_PER_TRACE):
    power_trace_col  = power_trace[:,col]
    power_trace_bar  = mean(power_trace_col)
    power_trace_sdev = sdev(power_trace_col, power_trace_bar) 

    mean_power_trace.append(power_trace_bar)
    sdev_power_trace.append(power_trace_sdev)

mean_power_trace = np.reshape(mean_power_trace, (NO_POINT_PER_TRACE))
sdev_power_trace = np.reshape(sdev_power_trace, (NO_POINT_PER_TRACE)) 


np.array_equal(mean_power_trace, mean_power_trace_1)
np.array_equal(sdev_power_trace, sdev_power_trace_1) 

# print(NO_POINT_PER_TRACE)
# print(mean_power_trace.shape)

True

True

In [75]:
# %%timeit
# power_trace_col  = np.ascontiguousarray(power_trace[:,1])
# power_trace_bar  = mean(power_trace_col)
# power_trace_col  = cuda.to_device(np.ascontiguousarray(power_trace[:,1]).flatten())
# power_trace_bar  = mean(power_trace_col)
# print(power_trace_col) 
# print(power_trace_bar) 
# print(np.ascontiguousarray(power_trace[:,1]).flatten().shape[0])
# print(power_trace[0,0])

# power_trace_bar  = mean(power_trace).flatten() 
# sdev_power_trace_cuda = np.zeros((power_trace.shape[1],1)).astype(float).flatten() 

# threadsperblock = (16,16) 
# blockspergrid_x = int(np.ceil(power_trace.shape[0]/threadsperblock[0]))
# blockspergrid_y = int(np.ceil(power_trace.shape[1]/threadsperblock[1]))
# blockspergrid   = (blockspergrid_x,blockspergrid_y)

# sdev_cuda[blockspergrid, threadsperblock](power_trace, power_trace_bar, sdev_power_trace_cuda) 



C:\Users\yoj20\anaconda3\lib\site-packages\numba\cuda\cudadrv\devicearray.py:790: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


5.07 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
# RUN CPA ATTACk

#hws_4all_bytes = []
corr_coeffs_4all_bytes = [[0]*NO_KEYS] * 16

def run_cpa_attack():
    for byte_num in trange(0,16):
        #hws_4all_keys = []
        corr_coeffs_4all_keys = [0] * NO_KEYS

        for kguess in trange(0, NO_KEYS):
            #hws_per_key = [] 

            hws = [0] * NO_PWR_TRC
            corr_coeffs = [0] * NO_POINT_PER_TRACE
            xor_byte_pos = CT_2_PREVCT_MAPPING[byte_num]

            for trc_row in range(NO_PWR_TRC):
                op1 = inverse_lookup(kguess, cipher_texts[trc_row, byte_num]) 
                op2 = cipher_texts[trc_row, xor_byte_pos] 
                hws[trc_row] = abs(HW[op1] - HW[op2])

            hws_bar = mean(hws)
            std_hws = sdev(hws,hws_bar)
            
            # Prepare Hamming Weights and Power Trace for Calculations
            A = np.subtract(hws,hws_bar).reshape((-1,1)) 
            B = np.subtract(power_trace, mean_power_trace) 
            
            # Covariance Calculation
            C = np.matmul(B.transpose(), A) 

            # Correlation Coefficient Calculation
            corr_coeffs = np.divide(C, np.multiply(sdev_power_trace, std_hws)) 
            corr_coeffs_4all_keys[kguess] = np.amax(corr_coeffs)

            # for trc_col in range(0, NO_POINT_PER_TRACE):
            #     power_trace_col  = power_trace[:,trc_col]
            #     num              = cov(hws,hws_bar,power_trace_col,mean_power_trace[trc_col])
            #     den              = sdev_power_trace[trc_col] * std_hws

            #     if (den != 0): 
            #         corr_coeff  = num / den
            #     else:
            #         corr_coeff = 0

            #     corr_coeffs[trc_col] = corr_coeff
            #     #hws_per_key.append(hws) 

            #hws_4all_keys.append(hws_per_key)
            #corr_coeffs_4all_keys[kguess] = max(corr_coeffs)
        
        #hws_4all_bytes.append(hws_4all_keys)
        corr_coeffs_4all_bytes[byte_num] = corr_coeffs_4all_keys


run_cpa_attack()

  0%|          | 0/16 [00:00<?, ?it/s]


C shape (1, 1)
corr_coeffs shape:  (1, 1)
[[0.02295841]]


  6%|▋         | 1/16 [00:03<00:58,  3.87s/it]

C shape (1, 1)
corr_coeffs shape:  (1, 1)
[[0.01455678]]


 12%|█▎        | 2/16 [00:07<00:54,  3.92s/it]

...

 88%|████████▊ | 14/16 [00:54<00:07,  3.84s/it]

C shape (1, 1)
corr_coeffs shape:  (1, 1)
[[-0.00603236]]


 94%|█████████▍| 15/16 [00:58<00:03,  3.85s/it]

C shape (1, 1)
corr_coeffs shape:  (1, 1)
[[-0.01371287]]


100%|██████████| 16/16 [01:01<00:00,  3.87s/it]


In [126]:

corr_coeffs_4all_bytes = np.array(corr_coeffs_4all_bytes)

for byte_no in range(0,16):
    result = np.where(corr_coeffs_4all_bytes[byte_no] == np.amax(corr_coeffs_4all_bytes[byte_no]))
    print(hex(result[0][0]))
    


0xb4
0xef
0xe5
0xcb
0x26
0x92
0x38
0xf6
0x23
0xe9
0x7f
0xcf
0x6f
0x8f
0x18
0x8e


In [120]:
#%%timeit
A = np.arange(1,10).reshape((3,3))
B = np.arange(1,4).reshape((-1,1)) 
print(A)
print(B)

C = np.matmul(A.transpose(),B) 
print(C)


[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1]
 [2]
 [3]]
[[30]
 [36]
 [42]]


In [134]:
B = np.arange(1,4)
F = np.arange(1,4)
G = np.divide(B,F)

print(B.shape)
print(F.shape)
print(G.shape)

(3,)
(3,)
(3,)


In [1]:


for j in range(4):
    for k in range(4):
        print("cipher[",k,'][',j,']','-> ciphertext[0][',(j*4+k),']' ) 

cipher[ 0 ][ 0 ] -> ciphertext[0][ 0 ]
cipher[ 1 ][ 0 ] -> ciphertext[0][ 1 ]
cipher[ 2 ][ 0 ] -> ciphertext[0][ 2 ]
cipher[ 3 ][ 0 ] -> ciphertext[0][ 3 ]
cipher[ 0 ][ 1 ] -> ciphertext[0][ 4 ]
cipher[ 1 ][ 1 ] -> ciphertext[0][ 5 ]
cipher[ 2 ][ 1 ] -> ciphertext[0][ 6 ]
cipher[ 3 ][ 1 ] -> ciphertext[0][ 7 ]
cipher[ 0 ][ 2 ] -> ciphertext[0][ 8 ]
cipher[ 1 ][ 2 ] -> ciphertext[0][ 9 ]
cipher[ 2 ][ 2 ] -> ciphertext[0][ 10 ]
cipher[ 3 ][ 2 ] -> ciphertext[0][ 11 ]
cipher[ 0 ][ 3 ] -> ciphertext[0][ 12 ]
cipher[ 1 ][ 3 ] -> ciphertext[0][ 13 ]
cipher[ 2 ][ 3 ] -> ciphertext[0][ 14 ]
cipher[ 3 ][ 3 ] -> ciphertext[0][ 15 ]


In [5]:


for j in range(16):
	post_row = j // 4
	post_col = j % 4

	pre_row = post_row
	pre_col = post_col - post_row

	if (pre_col < 0):
		pre_col += 4
    
	byte_id = pre_col * 4 + pre_row

	print('key[',byte_id,']',"  -> ct[",post_row,'][',post_col,']',"  ->ct_BP[",pre_row,'][',pre_col,']' ) 

    

key[ 0 ]   -> ct[ 0 ][ 0 ]   ->ct_BP[ 0 ][ 0 ]
key[ 4 ]   -> ct[ 0 ][ 1 ]   ->ct_BP[ 0 ][ 1 ]
key[ 8 ]   -> ct[ 0 ][ 2 ]   ->ct_BP[ 0 ][ 2 ]
key[ 12 ]   -> ct[ 0 ][ 3 ]   ->ct_BP[ 0 ][ 3 ]
key[ 13 ]   -> ct[ 1 ][ 0 ]   ->ct_BP[ 1 ][ 3 ]
key[ 1 ]   -> ct[ 1 ][ 1 ]   ->ct_BP[ 1 ][ 0 ]
key[ 5 ]   -> ct[ 1 ][ 2 ]   ->ct_BP[ 1 ][ 1 ]
key[ 9 ]   -> ct[ 1 ][ 3 ]   ->ct_BP[ 1 ][ 2 ]
key[ 10 ]   -> ct[ 2 ][ 0 ]   ->ct_BP[ 2 ][ 2 ]
key[ 14 ]   -> ct[ 2 ][ 1 ]   ->ct_BP[ 2 ][ 3 ]
key[ 2 ]   -> ct[ 2 ][ 2 ]   ->ct_BP[ 2 ][ 0 ]
key[ 6 ]   -> ct[ 2 ][ 3 ]   ->ct_BP[ 2 ][ 1 ]
key[ 7 ]   -> ct[ 3 ][ 0 ]   ->ct_BP[ 3 ][ 1 ]
key[ 11 ]   -> ct[ 3 ][ 1 ]   ->ct_BP[ 3 ][ 2 ]
key[ 15 ]   -> ct[ 3 ][ 2 ]   ->ct_BP[ 3 ][ 3 ]
key[ 3 ]   -> ct[ 3 ][ 3 ]   ->ct_BP[ 3 ][ 0 ]


In [6]:
HW = [bin(k).count("1") for k in np.arange(0,256)]

for i in range(10):
    print(HW[i ^ i+1])

1
2
1
3
1
2
1
4
1
2
